In [26]:
import numpy as np
from time import sleep
import tkinter as tk
from tkinter import ttk 
import serial.tools.list_ports
import serial
from datetime import datetime
import pandas as pd
import csv
from itertools import combinations
import keyboard
import h5py

dim = 5
len_of_window_to_preprocessing = 100
len_of_trial = 8

ser = serial.Serial('COM3', 9600)
ser.flushInput()

def compute_matrix_of_means(dim, len_of_window):
    data_array_for_compute_means = np.zeros((dim, dim, len_of_window), dtype="int64")
    for step_of_full_fill in range (len_of_window):
        data_array = np.zeros((dim, dim), dtype="int64")
        for rows in range(dim):
            data = ser.readline().decode('utf-8').strip()
            values = np.array([int(x) for x in data.split()])
            match values[0]:
                case 0: data_array[0] = values[1:]
                case 1: data_array[1] = values[1:]
                case 2: data_array[2] = values[1:]
                case 3: data_array[3] = values[1:]
                case 4: data_array[4] = values[1:]
        data_array_for_compute_means[:, :, step_of_full_fill] = data_array
    return np.mean(data_array_for_compute_means, axis=2) 
        
def add_new(a, b):
  a = np.delete(a, 0, axis=0)
  a = np.vstack((a, b))
  return a

def pairwise_subtractions(arr):
  valid_values = np.array(list((combinations(arr, 2))))
  values = valid_values[:, 0] - valid_values[:, 1]
  return values

def compute_matrix_of_relationships(data):
  new_full_matrix =  np.zeros((int(data.shape[0] * (data.shape[1] - 1) / 2), int(data.shape[0] * (data.shape[1] - 1) / 2)), dtype="float64")
  new_only_rows_matrix = np.zeros((data.shape[0], int(data.shape[0] * (data.shape[1] - 1) / 2)), dtype="float64")
  for j in range(int((data.shape[0] * (data.shape[1] - 1)) / 2)):
      for i in range(data.shape[0]):
          new_only_rows_matrix[i, :] = pairwise_subtractions(data[i, :])
      new_full_matrix[:, j] = pairwise_subtractions(new_only_rows_matrix[:, j])
  return new_full_matrix



def main_loop():
    global all_data 
    all_data = np.zeros((25, 12, len_of_trial, 1), dtype="float64")
    ser.flushInput()
    with open(txt_file, "w") as txtfile:
        labels = []
        while True:
            labels.append(4)
            ser.flushInput() 
            # Check for 'q' key press to break the loop
            if keyboard.is_pressed('q'):
                print('q')
                break
            one_sample_data = np.zeros((25, 12, len_of_trial), dtype="float64")
            
            for trial in range(len_of_trial):
                one_matrix = np.zeros((5, 5))
                for rows in range(dim):
                    data = ser.readline().decode('utf-8').strip()
                    #print(data)
                    values = np.array([int(x) for x in data.split()])
                    timestamp = datetime.now().strftime('%H:%M:%S.%f')
                    txtfile.write(f"{values},{timestamp}\n")
                    if len(values) == 6:
                        one_matrix[values[0]] = values[1:]
                one_matrix -= matrix_of_means
                
                one_matrix = pairwise_subtractions(one_matrix.reshape(25)).reshape(25, 12)
                
                one_sample_data[:, :, trial] = one_matrix
            all_data = np.concatenate((all_data, one_sample_data[:, :, :, np.newaxis]), axis=3)

        labels = np.array(labels)

    all_data = np.transpose(all_data, [3, 2, 0, 1])
    with h5py.File(hdf5_file, 'w') as hf:
        
        group = hf.create_group('mydataset')
        group.create_dataset('data', data=all_data, compression="gzip") 
        group.create_dataset('label', data=labels, dtype=np.int8)  
      

              
if __name__ == '__main__':
    
    txt_file = 'data_cucumber_4.txt'
    hdf5_file = 'data_cucumber_4.h5'
    matrix_of_means = compute_matrix_of_means(dim, len_of_window_to_preprocessing)
    print("Start")
    print(matrix_of_means)
    main_loop()
    print(all_data.shape)
    
    #print(all_data[:, :, :, 3])  
    with h5py.File(hdf5_file, 'r') as hf:
        read_data = hf['mydataset']['data']
        read_label = hf['mydataset']['label']  # [()] needed for scalar datasets

        print("Original Data Shape:", all_data.shape)

        print("Read Data Shape:", read_data.shape)
        print("Read Label Shape:", read_label.shape)
        print("Read Data:", read_data[:, :, 2, 3])
        print("Read Label:", read_label[:])
        
        #check if the data is identical.
        assert np.array_equal(all_data,read_data)

    


Start
[[4.000e-01 4.957e+01 1.300e-01 5.240e+00 8.770e+00]
 [4.000e-02 1.490e+01 5.300e-01 5.900e-01 6.400e-01]
 [3.360e+00 5.290e+00 2.000e-02 1.386e+01 0.000e+00]
 [1.600e-01 1.000e-02 2.000e-02 1.000e-02 0.000e+00]
 [1.639e+01 1.046e+01 1.734e+01 2.651e+01 8.000e-02]]
q
(1283, 8, 25, 12)
Original Data Shape: (1283, 8, 25, 12)
Read Data Shape: (1283, 8, 25, 12)
Read Label Shape: (1283,)
Read Data: [[  0.     0.     0.   ...   0.     0.     0.  ]
 [ 13.47  17.47  10.47 ...   7.47  30.47  84.47]
 [103.47 151.47 164.47 ... 118.47 115.47 108.47]
 ...
 [ 93.47  93.47  94.47 ... 105.47 101.47  99.47]
 [101.47 100.47  98.47 ... 111.47 110.47 114.47]
 [111.47 110.47 112.47 ... 106.47 110.47 115.47]]
Read Label: [4 4 4 ... 4 4 4]
